# Google Bigtable

> [Bigtable](https://cloud.google.com/bigtable) is a key-value and wide-column store, ideal for fast access to structured, semi-structured, or unstructured data. Extend your database application to build AI-powered experiences leveraging Bigtable's Langchain integrations.

This notebook goes over how to use an LLM agent to query [Bigtable](https://cloud.google.com/bigtable) with `BigtableGetInstancesAndTableSchemaTool` and `BigtableExecuteQueryTool`.


## Before You Begin

To run this notebook, you will need to do the following:

* [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
* [Enable the Bigtable API](https://console.cloud.google.com/flows/enableapi?apiid=bigtable.googleapis.com)
* [Create a Bigtable instance](https://cloud.google.com/bigtable/docs/creating-instance)
* [Create Bigtable access credentials](https://developers.google.com/workspace/guides/create-credentials)

After confirming access to the database in the runtime environment of this notebook, fill in the following values and run the cell before running example scripts.

### ☁ Set Your Google Cloud Project and Instance
`BigtableGetInstancesAndTableSchemaTool` and `BigtableExecuteQueryTool` requires an existing project and instance.

In [ ]:
# @markdown Specify a project and an instance for demo purposes.
PROJECT_ID = ""  # @param {type:"string"}
INSTANCE_ID = ""  # @param {type:"string"}

### 🦜🔗 Library Installation

Install the required libraries for Bigtable and LangChain integrations.

In [50]:
# Install required libraries
%pip install google-cloud-bigtable
%pip install --upgrade google-auth
%pip install langchain
%pip install -U langchain-google-genai
%pip install langgraph langchain-google-genai
%pip install -U langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


### 🔐 Authentication

Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

In [51]:
!gcloud auth application-default login
!gcloud config set project {PROJECT_ID}

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=BXLGEQopRCWsCxJrGjocTh2Th4wSsS&access_type=offline&code_challenge=slYz9KPQGBM2kuoNr3-e0DNSQepgQVu9gbVoTWZFnAk&code_challenge_method=S256


Credentials saved to file: [C:\Users\sophi\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "testproject-468100" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.
Updated property [core/project].


### 👍Initialize Bigtable Client, Create Tables and initialize the Tools

We will initialize two Bigtable clients:

1. **`admin_client`**: Used for administrative operations, such as creating and deleting tables.
2. **`data_client`**: Used for data operations, such as reading and writing rows.

We will also initialize the tools:

1. **`BigtableGetInstancesAndTableSchemaTool`**: To get instances and table schema.
2. **`BigtableExecuteQueryTool`**: To execute SQL query on Bigtable data.

We will also create two tables for query test:

1. **`hotels` table**: This table will be used for testing SQL queries.
2. **`fruits` table**: An unrelated table to demonstrate handling multiple tables.

Both tables will be cleaned up after the tests.


In [52]:
# Import necessary libraries
from google.cloud import bigtable
from google.cloud.bigtable import Client
from google.cloud.bigtable.data import BigtableDataClient
from langchain_google_bigtable.get_instances_and_table_schema_tool import BigtableGetInstancesAndTableSchemaTool
from langchain_google_bigtable.execute_query_tool import BigtableExecuteQueryTool

# initialize Bigtable clients
admin_client = bigtable.Client(project=PROJECT_ID, admin=True)
sync_data_client = BigtableDataClient(project=PROJECT_ID, admin=False)
instance = admin_client.instance(INSTANCE_ID)
print(f"Connected to Bigtable instance: {instance.instance_id}")

# Initialize tools
get_instances_and_table_schema_tool = BigtableGetInstancesAndTableSchemaTool(client=admin_client)
execute_query_tool = BigtableExecuteQueryTool(client=sync_data_client)
print("Tools initialized successfully.")

Connected to Bigtable instance: myinstance
Tools initialized successfully.


In [53]:
# Create a hotel table and insert data for demo purposes
instance = admin_client.instance(INSTANCE_ID)
hotels_table_id = "test-table-hotels"
hotels_table = instance.table(hotels_table_id)
column_families = {
    "cf": bigtable.column_family.MaxVersionsGCRule(1),
}

if not hotels_table.exists():
    hotels_table.create(column_families=column_families)

    # Define columns and data
    columns = ["id", "name", "location", "price_tier", "checkin_date", "checkout_date", "booked"]
    data = [
        [1, 'Hilton Basel', 'Basel', 'Luxury', '2024-04-20', '2024-04-22', False],
        [2, 'Marriott Zurich', 'Zurich', 'Upscale', '2024-04-14', '2024-04-21', False],
        [3, 'Hyatt Regency Basel', 'Basel', 'Upper Upscale', '2024-04-02', '2024-04-20', False],
        [4, 'Radisson Blu Lucerne', 'Lucerne', 'Midscale', '2024-04-05', '2024-04-24', False],
        [5, 'Best Western Bern', 'Bern', 'Upper Midscale', '2024-04-01', '2024-04-23', False],
        [6, 'InterContinental Geneva', 'Geneva', 'Luxury', '2024-04-23', '2024-04-28', False],
        [7, 'Sheraton Zurich', 'Zurich', 'Upper Upscale', '2024-04-02', '2024-04-27', False],
        [8, 'Holiday Inn Basel', 'Basel', 'Upper Midscale', '2024-04-09', '2024-04-24', False],
        [9, 'Courtyard Zurich', 'Zurich', 'Upscale', '2024-04-03', '2024-04-13', False],
        [10, 'Comfort Inn Bern', 'Bern', 'Midscale', '2024-04-04', '2024-04-16', False],
    ]

    # Insert data into the table
    mutations = []
    batcher = hotels_table.mutations_batcher(max_row_bytes=1024)
    for row in data:
        row_key = f"hotels#{row[0]}#{row[2]}#{row[1]}#{row[3]}"
        mutation = hotels_table.direct_row(row_key)
        for col, value in zip(columns, row):
            mutation.set_cell("cf", col.encode("utf-8"), str(value).encode("utf-8"))
        mutations.append(mutation)
    for mutation in mutations:
        batcher.mutate(mutation)

#create a fruits table and insert data for demo purposes
fruits_table_id = "test-table-fruits"
fruits_table = instance.table(fruits_table_id)
column_families = {
    "cf": bigtable.column_family.MaxVersionsGCRule(1),
} 
if not fruits_table.exists():
        fruits_table.create(column_families=column_families)

        columns = ["id", "name", "color", "price_per_kg", "in_stock"]
        data = [
            [1, 'Apple', 'Red', 3.5, True],
            [2, 'Banana', 'Yellow', 1.2, True],
            [3, 'Grapes', 'Purple', 2.8, False],
            [4, 'Orange', 'Orange', 2.0, True],
            [5, 'Blueberry', 'Blue', 4.0, False],
            [6, 'Strawberry', 'Red', 3.8, True],
            [7, 'Pineapple', 'Brown', 2.5, True],
            [8, 'Mango', 'Yellow', 3.0, False],
            [9, 'Watermelon', 'Green', 1.5, True],
            [10, 'Kiwi', 'Brown', 4.2, False],
        ]

        mutations = []
        batcher = fruits_table.mutations_batcher(max_row_bytes=1024)
        for row in data:
            row_key = f"fruits#{row[0]}#{row[1]}#{row[2]}"
            mutation = fruits_table.direct_row(row_key)
            for col, value in zip(columns, row):
                mutation.set_cell("cf", col.encode("utf-8"), str(value).encode("utf-8"))
            mutations.append(mutation)
        for mutation in mutations:
            batcher.mutate(mutation)

## Usage

This section demonstrates a sample usage of the Bigtable get table schema and execute query tool in the context of an LLM gemini agent.


`Note`: If you are using a Google model (e.g., `gemini-2.5-pro`), you need to set your Google API key. 
This key is required to authenticate and access Google's Generative AI services.


you can use the `create_react_agent` function to build a conversational agent.
This allows you to ask follow-up questions and interact dynamically with the Bigtable data.

### Steps:

1. Set the `GOOGLE_API_KEY` environment variable with your API key.
2. Use the `create_react_agent` function to initialize the agent.
3. Provide a prompt to guide the agent's behavior.
4. Ask questions interactively and process the responses.

In [54]:
# Set your Google API key
import os
os.environ["GOOGLE_API_KEY"]  = ""

In [55]:
# Create a conversational agent
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model="gemini-2.5-pro",
    tools=[get_instances_and_table_schema_tool, execute_query_tool],
    prompt="You are a helpful assistant. Make sure to process the data retrieved from the Bigtable execute query tool to answer user questions effectively."
)

# Ask the agent a question about the hotels table
response = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": (
                "I want to figure out what hotels are luxurious. Don't ask me for any other information. Use the tools at your disposal to get the information you need."
            )
        }
    ]
})

# Print the agent's response to debug - the result is verbose and contains all the steps the agent took to arrive at the final answer
# print(response)

In [56]:
# Import the AiMessage class
from langchain_core.messages.ai import BaseMessage


# Print the final thought and agent return.
final_message: BaseMessage = response["messages"][-1]
print("Final Thought:")
final_message.pretty_print()
print("\n")


# Print each thought.
print("All Steps:\n")
for i, message in enumerate(response["messages"], start=1):
    message: BaseMessage
    print(f"Step {i}:")
    message.pretty_print()
    print("\n")


Final Thought:
================================== Ai Message ==================================

Of course. The luxurious hotels are the Hilton Basel, the InterContinental Geneva, the Hyatt Regency Basel, and the Sheraton Zurich.


All Steps:

Step 1:
================================ Human Message =================================

I want to figure out what hotels are luxurious. Don't ask me for any other information. Use the tools at your disposal to get the information you need.


Step 2:
================================== Ai Message ==================================
Tool Calls:
  GetBigtableInstancesAndTableSchema (c6ed80bf-38d4-4f59-b313-dd0925481378)
 Call ID: c6ed80bf-38d4-4f59-b313-dd0925481378
  Args:


Step 3:
================================= Tool Message =================================
Name: GetBigtableInstancesAndTableSchema

{"myinstance": {"hotel1": {"column_families": ["cf"], "possible_columns": ["cf['price_tier']", "cf['checkout_date']", "cf['location']", "cf['id']",

### Cleanup

After the tests, the `hotels` and `fruits` tables should be deleted to clean up resources.

In [57]:
# Cleanup tables
def delete_table(table):
    if table.exists():
        table.delete()
        print(f"Table '{table.table_id}' deleted successfully.")
    else:
        print(f"Table '{table.table_id}' does not exist.")

delete_table(hotels_table)
delete_table(fruits_table)

Table 'test-table-hotels' deleted successfully.
Table 'test-table-fruits' deleted successfully.
